In [5]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.preprocessing import LabelEncoder

from keras import layers
from keras import models
from keras import losses

In [6]:
dateColumnNames = [
    'contact_date',
    'Glycemie_der_date',
    'HbA1c_der_date',
    'der_date_poids',
    'der_date_taille',
    'first_contact_date'
]

dfView = pd.read_csv('PatientsHTA.zip',nrows=1)
df = pd.read_csv('PatientsHTA.zip',engine='c',parse_dates=dateColumnNames)
df = df[df.first_contact_date.values <= df.contact_date.values]

# Suppression des lignes trop peu nombreuses

Nous souhaitons faire un apprentissage en utilisant la dimension temporelle comme filtre pour le CNN. Pour ça il faut donc que nous ayons plusieurs entrée. Avant de commencer à traîter les données, nous supprimons toutes les personnes qui n'ont pas rendu visite assez souvent à leur médecin. Ainsi, par le biais de ```person_id```, nous avons choisi arbitrairement que pour être utile à l'apprentissage, il faut au moins 4 visites par patient, soient toutes les lignes dont le ```person_id```est contenu plus de 4 fois dans tout le jeu de données.

In [7]:
valueCounts = df.person_id.value_counts()
dfEnought = df[df.person_id.isin(valueCounts[valueCounts.values >= 4].index)]

# Suppression des colonnes innutiles

## Suppression de la colonne ```age_now```

Nous pouvons supprimer la colonne ```age_now``` car les données qu'elle contient sont identiques à celles de la colonne ```year_of_birth```.



In [8]:
dfWithoutAgeNow = dfEnought.drop('Age_now', axis='columns')

## Suppression de la colonne ```contact_id```

En effet, la colonne ```contact_id``` ne représente aucun intérêt pour l'apprentissage car elle ne contient aucun information à même d'influer sur la prédiction.

In [9]:
dfWithoutContactID = dfWithoutAgeNow.drop('contact_id',axis='columns')

## Suppression des noms de médicaments

In [10]:
dfGroupedByMoleculeLabel = dfWithoutContactID.groupby('product_atc_code')[['molecule_label','short_name','long_name','Classe','product_atc']].count()
print(dfGroupedByMoleculeLabel)

                  molecule_label  short_name  long_name  Classe  product_atc
product_atc_code                                                            
C02AC06                     4239        4239       4239    4239         4239
C03BX03                      924         924        924     924          924
C03CA01                      869         869        869     869          869
C03DA01                      186         186        186     186          186
C03DA04                       23          23         23      23           23
C03EA04                       75          75         75      75           75
C03EB01                       10          10         10      10           10
C07AA05                        2           2          2       2            2
C07AB03                     6358        6358       6358    6358         6358
C07AB04                       39          39         39      39           39
C07AB07                     6687        6687       6687    6687         6687

Nous voyons qu'il existe différentes colonnes dont le but est de désigner le médicament prescrit lors de la visite, or nous n'avons besoin que d'une seule colonne garder cette information. De ce fait, nous avons choisi de garder ```product_atc_code```.

In [11]:
dropColumnNames = dfGroupedByMoleculeLabel.columns.to_list()
dfWithATCCode = dfWithoutContactID.drop(dropColumnNames, axis='columns')
dfWithATCCode

,person_id,specialty_label,contact_date,cip,dosage_1,dose_1,dose_2,product_atc_code,box,quantity,...,HbA1c_der_date,HbA1c_der_mesure,gender_code,Age_presc,year_of_birth,Poids,der_date_poids,Taille,der_date_taille,first_contact_date
1,263659.0,Médecin généraliste,2014-09-24,3.400960e+12,20.0,240.0,24.0,C03CA01,1.0,4.00,...,NaT,NaN,M,96.0,1918.0,NaN,NaT,NaN,NaT,1998-04-02
2,263659.0,Médecin généraliste,2014-09-12,3.400960e+12,20.0,240.0,24.0,C03CA01,1.0,4.00,...,NaT,NaN,M,96.0,1918.0,NaN,NaT,NaN,NaT,1998-04-02
3,263659.0,Médecin généraliste,2015-03-26,3.400960e+12,20.0,240.0,24.0,C03CA01,1.0,4.00,...,NaT,NaN,M,97.0,1918.0,NaN,NaT,NaN,NaT,1998-04-02
4,263659.0,Médecin généraliste,2015-06-05,3.400960e+12,20.0,240.0,24.0,C03CA01,1.0,4.00,...,NaT,NaN,M,97.0,1918.0,NaN,NaT,NaN,NaT,1998-04-02
6,25182917.0,Médecin généraliste,2013-03-27,3.400960e+12,300.0,16800.0,56.0,C09XA02,2.0,1.00,...,NaT,NaN,M,63.0,1950.0,NaN,NaT,NaN,NaT,2011-04-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50625,26636453.0,Médecin généraliste,2016-11-16,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,0.75,...,2017-10-03,5.60,M,79.0,1937.0,83.9,2018-02-21,NaN,NaT,2012-02-14
50627,18889430.0,Médecin généraliste,2013-02-01,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,0.50,...,2017-05-15,8.01,M,85.0,1928.0,NaN,NaT,NaN,NaT,2007-12-02
50629,2222336.0,Médecin généraliste,2013-07-01,3.400940e+12,500.0,15000.0,30.0,C03CA01,2.0,0.25,...,NaT,NaN,F,78.0,1935.0,NaN,NaT,NaN,NaT,2006-02-23
50631,11363518.0,Médecin généraliste,2013-01-24,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,0.25,...,NaT,NaN,M,80.0,1933.0,NaN,NaT,NaN,NaT,1998-01-27


## Suppression des colonnes ```'*der*'```

Les colonnes ```'*der*'``` contiennent la dernière donnée. Cette donnée peut être récupérée grâce à la date de la visite et aux valeurs mesurées. Par exemple, il n'est pas nécessaire d'avoir une colonne ```der_date``` ou ```der_mesure```. Les données de ces deux types de colonnes peuvent être récupérées grâce à la ligne qui correspond à la dernière date de la mesure, que nous pouvons trouver grâce à la colonne ```contact_date```.

In [12]:
derColumnNames = []

for c in dfWithATCCode.columns:
    if ('der_date' in c) or ('der_mesure' in c):
        derColumnNames.append(c)

dfWithoutDer = dfWithATCCode.drop(derColumnNames,axis='columns')

# Traîtement des données

## Conversion des données

### Ajout du temps entre chaque visite (ce que l'on veut prédire)

In [19]:
dfWithoutDer.sort_values(['person_id','contact_date'], ascending=[True,True], inplace=True)

# wait_time_list = list()
# for k, v in dfWithoutDer[:10].iterrows():
#     if k > 0:
#         k1 = dfWithoutDer.iloc[k-1] if dfWithoutDer.iloc[k-1] else None 
#         if k1 and k1.person_id and k1.person_id == v.person_id:
#             wait_time_list.append(  v.contact_date - k1.contact_date )
#         else:
#             wait_time_list.append( v.contact_date - v.first_contact_date )
#     else:
#         wait_time_list.append( v.contact_date - v.first_contact_date )

# dfWithoutDer.append( {'wait_time': wait_time_list }, ignore_index=True )

# print(dfWithoutDer)

#wait_time = dfWithoutDer.contact_date - dfWithoutDer.first_contact_date

On passe en semaine pour avoir un
wait_time = np.floor(wait_time.dt.days / 7)

dfWithTime = dfWithoutDer.drop('first_contact_date',axis='columns')
dfWithTime['wait_time'] = wait_time

11975
person_id                                           291
specialty_label                     Médecin généraliste
contact_date                        2016-10-28 00:00:00
cip                                         3.40095e+12
dosage_1                                            2.5
dose_1                                               75
dose_2                                               30
product_atc_code                               C07AB07 
box                                                   6
quantity                                              2
frequency_label                                    Jour
duration                                             84
Traitement_Autres_A10_dep_201701                    OUI
Traitement_Insulines_dep_201701                     OUI
Pulse                                                72
Tension Diastolique                                 130
Tension Systolique                                   60
Glycemie_prescription                     

AttributeError: 'Timestamp' object has no attribute 'person_id'

In [ ]:
dfWithoutDer

### Encodage des valeurs non numériques

In [56]:
specialtyEncoder = LabelEncoder()
ATCCodeEncoder = LabelEncoder()
frequencyLabelEncoder = LabelEncoder()
traitementAutresLabelEncoder = LabelEncoder()
traitementInsulineLabelEncoder = LabelEncoder()
genderEncoder = LabelEncoder()

dfWithTime.specialty_label = specialtyEncoder.fit_transform(dfWithTime.specialty_label)
dfWithTime.product_atc_code = ATCCodeEncoder.fit_transform(dfWithTime.product_atc_code)
dfWithTime.frequency_label = frequencyLabelEncoder.fit_transform(dfWithTime.frequency_label.astype(str))
dfWithTime.Traitement_Autres_A10_dep_201701 = traitementAutresLabelEncoder.fit_transform(dfWithTime.Traitement_Autres_A10_dep_201701.astype(str))
dfWithTime.Traitement_Insulines_dep_201701 = traitementInsulineLabelEncoder.fit_transform(dfWithTime.Traitement_Insulines_dep_201701.astype(str))
dfWithTime.gender_code = ATCCodeEncoder.fit_transform(dfWithTime.gender_code)


### Conversion en ```TimeSeries```

Nous définissons simplement le nouvel index comme la colonne donnant l'intervalle de temps entre chaque visite.

In [58]:
ts = dfWithTime.set_index(['person_id','contact_date'])
ts

specialty_label           cip  dosage_1  dose_1  \
person_id  contact_date                                                    
291.0      2016-10-28                  2  3.400950e+12       2.5    75.0   
           2016-11-04                  2  3.400950e+12       2.5    75.0   
           2016-11-21                  2  3.400950e+12       2.5    75.0   
           2017-01-16                  2  3.400950e+12       2.5    75.0   
           2017-11-20                  2  3.400950e+12       2.5    75.0   
...                                  ...           ...       ...     ...   
34823672.0 2017-04-15                  2  3.400950e+12     100.0  9000.0   
           2017-06-26                  2  3.400950e+12     100.0  9000.0   
           2017-08-30                  2  3.400950e+12     100.0  9000.0   
           2017-09-11                  2  3.400950e+12     100.0  9000.0   
           2018-08-02                  2  3.400950e+12     100.0  9000.0   

                         dose_2  product_atc_code  box  quantity  \
person_id  contact_date                                            
291.0      2016-10-28      30.0                10  6.0       2.0   
           2016-11-04      30.0                10  3.0       1.0   
           2016-11-21      30.0                10  3.0       1.0   
           2017-01-16      30.0                10  3.0       1.0   
           2017-11-20      30.0                10  3.0       1.0   
...                         ...               ...  ...       ...   
34823672.0 2017-04-15      90.0                 8  1.0       1.0   
           2017-06-26      90.0                 8  1.0       1.0   
           2017-08-30      90.0                 8  1.0       1.0   
           2017-09-11      90.0                 8  1.0       1.0   
           2018-08-02      90.0                 8  1.0       1.0   

                         frequency_label  duration  ...  Tension Diastolique  \
person_id  contact_date                             ...                        
291.0      2016-10-28                  0      84.0  ...                130.0   
           2016-11-04                  0      84.0  ...                142.0   
           2016-11-21                  0      84.0  ...                140.0   
           2017-01-16                  0      84.0  ...                125.0   
           2017-11-20                  0      84.0  ...                115.0   
...                                  ...       ...  ...                  ...   
34823672.0 2017-04-15                  0      84.0  ...                140.0   
           2017-06-26                  0      56.0  ...                130.0   
           2017-08-30                  0      56.0  ...                130.0   
           2017-09-11                  0      56.0  ...                140.0   
           2018-08-02                  0      56.0  ...                130.0   

                         Tension Systolique  Glycemie_prescription  \
person_id  contact_date                                              
291.0      2016-10-28                  60.0                    NaN   
           2016-11-04                  65.0                    NaN   
           2016-11-21                  75.0                    NaN   
           2017-01-16                  70.0                    NaN   
           2017-11-20                  60.0                    NaN   
...                                     ...                    ...   
34823672.0 2017-04-15                  60.0                    NaN   
           2017-06-26                  70.0                    NaN   
           2017-08-30                  70.0                    NaN   
           2017-09-11                  80.0                    NaN   
           2018-08-02                  70.0                    NaN   

                         HbA1c_prescription  gender_code  Age_presc  \
person_id  contact_date                                               
291.0      2016-10-28                   NaN            1 

# Prédiction

## Création des données d'entraînement/test

In [105]:
yColumNames = ['product_atc_code','wait_time']

xDf = ts.drop(yColumNames,axis='columns')

# Pour l'instant on ne prédit que l'atc code et c'est déjà un sacré bordel
yDf = ts.product_atc_code#loc[:,yColumNames]

xList,yList = [],[]
sliceNumber = 4

for i,_ in ts.groupby('person_id'):
    # Mme ZERHAOUI a dit qu'il fallait transposer, je transpose
    xList.append(xDf.loc[i][:sliceNumber].to_numpy().astype('float32').transpose())
    yList.append(yDf.loc[i][:sliceNumber].to_numpy().astype('float32'))

xData = np.array(xList).reshape((len(xList),xList[0].shape[0],sliceNumber))
yData = np.array(yList).reshape((sliceNumber,len(yList)))#,yList[0].shape[1]))

In [60]:
print(f"{xData.shape}, {yData.shape}")

(4185, 21, 4), (4, 4185)


In [61]:
# On crée un jeu d'entraînement et de test :
#  - l'entraînement se fait sur 80% du jeu de donnée total
#  - le test se fait donc sur 20%

trainUse = int(xData.shape[0] * 80 / 100)
testUse = xData.shape[0] - trainUse

xTrain, xTest, yTrain, yTest = xData[:trainUse],xData[-testUse:],yData[:,:trainUse],yData[:,-testUse:]

#xTrain = xTrain.reshape(xTrain.shape[0],1,xTrain.shape[1],xTrain.shape[2])
#xTest = xTest.reshape(xTest.shape[0],1,xTest.shape[1],xTest.shape[2])

## Création du modèle

In [62]:
print(f"{xTrain.shape}, {yTrain.shape}")

(3348, 21, 4), (4, 3348)


In [71]:
# Alors c'est totalement au hasard, j'ai essayé de suivre ce qu'il y avait dans la partie 'keras exemple' du lien que farida nous a donné :
# cf : https://towardsdatascience.com/how-to-use-convolutional-neural-networks-for-time-series-classification-56b1b0a07a57
model = models.Sequential()
kernelNumber = 20
model.add(layers.Conv1D(kernelNumber,21,activation='tanh',input_shape=(xTrain.shape[1],sliceNumber)))
model.add(layers.MaxPool1D(1))
model.add(layers.Flatten())
model.add(layers.Dense(1,activation='sigmoid'))

In [72]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 1, 20)             1700      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1, 20)             0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 1,721
Trainable params: 1,721
Non-trainable params: 0
_________________________________________________________________


In [103]:
# model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [104]:
# pour utiliser sparse cross entropy apparement il faut des valeurs entre 0 et 1 pourtant sur ce lien il l'utilise pour prédire les données mniste entre 0 et 9 'https://www.machinecurve.com/index.php/2019/10/06/how-to-use-sparse-categorical-crossentropy-in-keras/'
for i in range(yTrain.shape[0]):
    print(f"--- Training: {i+1}/{yTrain.shape[0]} ---")
    model.fit(xTrain,yTrain[i],epochs=20, validation_data=(xTest,yTest[i]))

--- Training: 1/4 ---
Epoch 1/20
105/105 [==============================] - 2s 9ms/step - loss: nan - accuracy: 0.1219 - val_loss: nan - val_accuracy: 0.1159
Epoch 2/20
105/105 [==============================] - 1s 5ms/step - loss: nan - accuracy: 0.1258 - val_loss: nan - val_accuracy: 0.1159
Epoch 3/20
105/105 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.1167 - val_loss: nan - val_accuracy: 0.1159
Epoch 4/20
105/105 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.1097 - val_loss: nan - val_accuracy: 0.1159
Epoch 5/20
105/105 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.1210 - val_loss: nan - val_accuracy: 0.1159
Epoch 6/20
105/105 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.1167 - val_loss: nan - val_accuracy: 0.1159
Epoch 7/20
105/105 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.1196 - val_loss: nan - val_accuracy: 0.1159
Epoch 8/20
105/105 [======

In [74]:
model.predict(xTest[5].reshape((1,21,4)))

(3348, 21, 4), (4, 3348)
